# Plotting

*Author: Alex Lewandowski; Alaska Satellite Facility*

Rerun the plots in the Perform *Mintpy Time Series Analysis notebook* without rerunning the time series

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
from ipyfilechooser import FileChooser
from pathlib import Path
import sys

from mintpy.cli import view, tsview, plot_network, plot_transection, plot_coherence_matrix
import mintpy.plot_coherence_matrix
import mintpy.utils
import numpy as np
import opensarlab_lib as osl

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

In [ ]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

config_path = Path(fc.selected)
mint_path = config_path.parent
plot_path = mint_path / 'plots'
inputs_path = mint_path / 'inputs'

velocity_path = mint_path/'velocity.h5'
geotiff_path = mint_path / 'GeoTiffs'
disp_path = geotiff_path / 'displacement_maps'
unwrapped_path = disp_path / 'unwrapped'
unwrapped_path.mkdir(parents=True, exist_ok=True)
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))

**If you have both tropospheric delay corrected and uncorrected time series available, select which to plot**

- If you have run the time series more than once, with tropospheric correction both on and off, you will have time series saved for both configurations
- `timeseries_demErr.h5` is the time series not corrected for tropospheric delay
- `timeseries_ERA5.h5` is the time series corrected for tropospheric delay

In [ ]:
if len(ts_demErr_path) > 1:
    ts_select_option = osl.select_parameter(
        [
            ts_demErr_path[0],
            ts_demErr_path[1]
        ],
        description='Select the time series file you wish to plot:'
    )
    display(ts_select_option)
else:
    ts_demErr_sel_path = ts_demErr_path[0]

In [ ]:
if len(ts_demErr_path) > 1:
    ts_demErr_sel_path = ts_select_option.value

---
## 2. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with osl.work_dir(mint_path):
    scp_args = f'{inputs_path}/ifgramStack.h5 --cmap-vlist 0.2 0.7 1.0'
    plot_network.main(scp_args.split())

**Plot the velocity map**

Note: The min/max values in this plot should be adjusted to your data set. 

The output of the following call will show a data range. You can run the cell once to collect this information and re-run it after adjustments are made.

- You should update `vmin` and `vmax` to match your data range
- The `vmin` and `vmax` values that you assign will be used for additional plots in this notebook
- If you wish to set 0 deformation to the center of the color scale (green), adjust your min/max values so they are symmetrical
    - For example, given a data range of `[-10.691118, 13.904866]`, appropriate symmetrical min/max values would be `[-14.0, 14.0]` 

In [ ]:
colormap_select = osl.select_parameter(
    [
        "Display red for uplift, and blue for subsidence",
        "Display blue for uplift, and red for subsidence"
    ]
)
display(colormap_select)

In [ ]:
%matplotlib inline
cmap = 'jet' if 'red for uplift' in colormap_select.value else 'jet_r'

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(mint_path/'velocity.h5', mask_path=mint_path/'maskTempCoh.h5', bottom_percentile=0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

scp_args = f'{mint_path}/velocity.h5 velocity -v {vel_vmin} {vel_vmax} --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png --update --colormap {cmap}'
view.main(scp_args.split())

---
## 3. Plot the unwrapped inverted timeseries

In [ ]:
inv_vmin, inv_vmax = util.get_mintpy_vmin_vmax(ts_demErr_sel_path, mask_path=mint_path/'maskTempCoh.h5', bottom_percentile=0.0)

# uncomment the following 2 lines to manually set the vmin and vmax values
# inv_vmin = -2.0
# inv_vmax = 2.0

scp_args = f'{ts_demErr_sel_path} --notick --noaxis -v {inv_vmin} {inv_vmax} --dpi 600 --figsize 15 15 --outfile {unwrapped_path}/unwrapped_inverted_ts.png'
view.main(scp_args.split())

---
## 3. Plot a Motion Transect

**Select two points to define a transect on an interactive plot**

In [ ]:
%matplotlib widget

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(velocity_path, mask_path=mint_path/'maskTempCoh.h5', bottom_percentile=0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

data, vel_info = mintpy.utils.readfile.read(velocity_path)
mask = np.ma.masked_where(data==0, data)
data = mask.filled(fill_value=np.nan)
line = osl.LineSelector(data, figsize=(9, 9), cmap='jet', vmin=vel_vmin/100, vmax=vel_vmax/100)

**Plot the selected motion transect**

In [ ]:
from IPython.display import Markdown

geotrans = (
    float(vel_info['X_FIRST']),
    float(vel_info['X_STEP']),
    0.0, 
    float(vel_info['Y_FIRST']), 
    0.0, 
    float(vel_info['Y_STEP'])
)

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

try:
    pnt_1 = geolocation(line.pnt1[0][0], line.pnt1[0][1], geotrans)
    pnt_2 = geolocation(line.pnt2[0][0], line.pnt2[0][1], geotrans)
    print(f"point 1: {pnt_1}")
    print(f"point 2: {pnt_2}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if a line was not selected.</text>'))

scp_args = f'{mint_path}/velocity.h5 -v {vel_vmin} {vel_vmax} --start-lalo {pnt_1[1]} {pnt_1[0]} --end-lalo {pnt_2[1]} {pnt_2[0]} --outfile x'

with osl.work_dir(plot_path):
    # let MintPy cleanup its old subplots 
    # this has been fixed in MintPy 1.5.3 but is required for 1.5.1
    plot_transection.main(scp_args.split()[:-2] + ['--noverbose', '--nodisplay'])
    # plot transection after removing old subplots
    %matplotlib inline
    plot_transection.main(scp_args.split())

---
## 4. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget

tsview.main(
    [
        str(ts_demErr_sel_path),
        f'-d={mint_path}/inputs/geometryGeo.h5', 
        f'-o={mint_path}/displacement_ts', 
        f'--outfile={mint_path}/displacement_ts.pdf'
    ]
)